In [1]:
from forest.forest_subset_features.class_forest import CustomEnsembleClassifier
import pandas as pd
import time
import os

In [2]:
from sklearn.model_selection import train_test_split

#data_seismic = pd.read_csv("datasets/seismic/seismic_bin.csv")
data_test = pd.read_csv("datasets/example_datasets/stacked.csv")

data = data_test


X = data.drop(columns=['y'])  # All columns except the target
y = data['y']                 # Only the target column

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=100, stratify=y, random_state=42)

stacked_train = pd.concat([y_train, X_train], axis=1, ignore_index=False)
stacked_test = pd.concat([y_test, X_test],axis=1, ignore_index=False)

train_data = stacked_train
test_data = stacked_test

#feature_columns = train_data.columns[1:]

feature_columns = X_train.columns
#print(feature_columns)

In [ ]:
tree_kwargs = dict(depth=8, criterion='gini', target_label='y', features=list(range(X_train.shape[1])))

ensemble = CustomEnsembleClassifier(n_estimators=2, tree_kwargs=tree_kwargs, random_state=42, cores_to_use=6)

In [4]:
ensemble.parallel_fit(X_train, y_train)

TypeError: 'NoneType' object is not iterable

In [ ]:
y_pred = ensemble.predict(X_test)

/home/drood/Obsidian/Files/Bachelorarbeit/rlrf_my_try/rolling_lookahead_dt_restructured/rolling_lookahead_dt_pulp/oct/optimal_tree_pulp.py:367: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = np.array(i[P])
/home/drood/Obsidian/Files/Bachelorarbeit/rlrf_my_try/rolling_lookahead_dt_restructured/rolling_lookahead_dt_pulp/oct/optimal_tree_pulp.py:367: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = np.array(i[P])


In [ ]:
res = pd.DataFrame()
print(y_test)
res['y'] = y_test
res['prediction'] = y_pred
print(res)
unique_values = res['prediction'].unique()
print(unique_values)

94     2
42     1
52     1
29     1
106    2
      ..
27     1
123    3
77     2
109    2
38     1
Name: y, Length: 100, dtype: int64
     y  prediction
94   2           1
42   1           1
52   1           1
29   1           2
106  2           1
..  ..         ...
27   1           1
123  3           3
77   2           2
109  2           2
38   1           1

[100 rows x 2 columns]
[1 2 3]


In [ ]:
import numpy as np
import pandas as pd
import copy
import logging
from typing import Union, List


from sklearn.model_selection import train_test_split

#data_seismic = pd.read_csv("datasets/seismic/seismic_bin.csv")
data_test = pd.read_csv("datasets/example_datasets/stacked.csv")

data = data_test


X = data.drop(columns=['y'])  # All columns except the target
y = data['y']                 # Only the target column

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=100, stratify=y, random_state=42)

train = pd.concat([y_train, X_train], axis=1, ignore_index=False)
test = pd.concat([y_test, X_test],axis=1, ignore_index=False)

df = pd.concat([train_data, test_data])
K = sorted(list(set(df.y))) #K is a sorted list of all unique class labels (target categories) present in the combined training and test data

all_features_list = [int(i) for i in list(train.loc[:, train.columns != 'y'].columns)] #used to be P
features_orig_dataset = len(all_features_list)

K = sorted(list(set(df.y)))

# --- Choose subset of features for THIS TREE/BLOCK (random forest style) ---
amount_X_consider = max(1, int(np.sqrt(features_orig_dataset)))
rng = np.random.RandomState(None)
selected_features = rng.choice(all_features_list, size=amount_X_consider, replace=False).tolist()

    # Use only the selected features for this tree
train = train[['y'] + selected_features]
test = test[['y'] + selected_features]

P = selected_features  # list of indices
P = list(P)

#features = df.iloc[:, 1:] 
features = data.loc[:, data.columns != 'y']

selected_feature_cols = np.random.choice(features.columns, size=amount_X_consider, replace=True)
selected_features = features[selected_feature_cols]

KeyError: '[98, 11, 47, 6, 29, 31, 64, 48, 94, 63, 89] not in index'

In [ ]:
for feature in P:
    print(feature)

0
1
2
3
4
5
6
7
8
9
10


In [ ]:
def gini_index(arr: np.array,
               instance_size: int,
               K: list,
               y_idx: int = 0,
               weighted: bool = True):
    """

    :param arr:
    :param instance_size:
    :param K:
    :param y_idx:
    :param weighted:
    :return:
    """
    sum_ = 0
    for k in K:
        sum_ += np.power(len(arr[np.where(arr[:, y_idx] == k)]) / len(arr), 2)
    sum_ = 1 - sum_
    if weighted:
        sum_ = (len(arr) / instance_size) * sum_
    return sum_

In [ ]:
def calculate_gini_old(data: pd.DataFrame,
                   P: list,
                   K: list,
                   nodes: dict) -> dict:
    """

    :param data: pd.DataFrame — The dataset
    :param P: list Indices of features to consider
    :param K: lsit Unique class labels
    :param nodes: dict — Contains: "leaf_nodes": list of leaf node identifiers. leaf_nodes_path": dict mapping each leaf node to a path (list of values for features).
    :return:
    """
    df_arr = np.array(data)
    n = len(data) # total number of instances.
    gini_dict = dict()
    for leaf_ in nodes["leaf_nodes"]:
        temp = dict() # store Gini values for this leaf
        first_var = nodes["leaf_nodes_path"][leaf_][0] #{'leaf_nodes': [4, 5, 6, 7], 'leaf_nodes_path': {4: [1, 1], 5: [1, 0], 6: [0, 1], 7: [0, 0]}}
        second_var = nodes["leaf_nodes_path"][leaf_][1] #first_var, second_var — the first two values in the path to this leaf (used as feature values for filtering)
        for feature_i in P:
            arr = df_arr[np.where((df_arr[:, feature_i] == first_var))] #Selects rows where feature feature_i equals first_var.

            for feature_j in P:
                arr_2 = arr[np.where(arr[:, feature_j] == second_var)] #Further filters to rows where feature feature_j equals second_var
                if len(arr_2) > 0: #Calculate Gini index for arr_2 (so for all rows matching (1, 0; having/not having feature i) the decision variables)
                    temp[feature_i, feature_j] = gini_index(arr=arr_2,
                                                            instance_size=n,
                                                            K=K,
                                                            weighted=True)
        gini_dict[leaf_] = copy.deepcopy(temp)
        del temp
        del arr

    return gini_dict

In [ ]:
def generate_nodes(depth: int) -> list:
    """

    :param depth:
    :return:
    """
    nodes = list(range(1, int(round(2 ** (depth + 1)))))
    parent_nodes = nodes[0: 2 ** (depth + 1) - 2 ** depth - 1]
    leaf_nodes = nodes[-2 ** depth:]
    return parent_nodes, leaf_nodes

In [ ]:
leaf_nodes_path = {4: [1, 1], #dict for all leafes in depth 2 tree and der respective split conditions
                       5: [1, 0],
                       6: [0, 1],
                       7: [0, 0]}
depth = 2
parent_nodes, leaf_nodes = generate_nodes(depth) # (for depth 2) returns [1,2,3] and [4,5,6,7]

nodes = dict()
nodes["leaf_nodes"] = leaf_nodes
nodes["leaf_nodes_path"] = leaf_nodes_path


In [ ]:
coef_dict = calculate_gini_old(data=train,
                                    P=P,
                                    K=K,
                                    nodes=nodes)

print(coef_dict)

{4: {(0, 0): np.float64(0.0), (0, 1): np.float64(0.0), (0, 2): np.float64(0.0), (0, 3): np.float64(0.0), (0, 4): np.float64(0.0), (0, 6): np.float64(0.0), (0, 7): np.float64(0.0), (0, 8): np.float64(0.0), (0, 9): np.float64(0.0), (0, 10): np.float64(0.0), (1, 0): np.float64(0.0), (1, 1): np.float64(0.02775941837409121), (1, 2): np.float64(0.0), (1, 3): np.float64(0.0), (1, 4): np.float64(0.0056179775280898875), (1, 7): np.float64(0.0), (1, 8): np.float64(0.0), (2, 0): np.float64(0.0), (2, 1): np.float64(0.0), (2, 2): np.float64(0.026484751203852328), (2, 4): np.float64(0.0), (2, 5): np.float64(0.0), (2, 6): np.float64(0.0), (2, 7): np.float64(0.0), (3, 0): np.float64(0.0), (3, 1): np.float64(0.0), (3, 3): np.float64(0.059925093632958795), (3, 5): np.float64(0.0), (3, 7): np.float64(0.0), (3, 8): np.float64(0.014044943820224719), (3, 9): np.float64(0.0), (3, 10): np.float64(0.0), (4, 0): np.float64(0.0), (4, 1): np.float64(0.0056179775280898875), (4, 2): np.float64(0.0), (4, 4): np.floa